In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# 📁 Paths
BASE_DIR = "C:/Users/MANJU/Desktop/FYP_Diabetes/data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 📐 Parameters
IMG_HEIGHT, IMG_WIDTH = 300, 300  # EfficientNetB3 image size
BATCH_SIZE = 32
EPOCHS = 50

# 🔁 Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # ✅ Binary Classification
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# ⚙️ Load EfficientNetB3 base model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze base layers

# 🎯 Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # ✅ Binary output

model = Model(inputs=base_model.input, outputs=output)

# 🔧 Compile
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# 📦 Callbacks
checkpoint = ModelCheckpoint("best_efficientnetb3_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 🚀 Train
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint, early_stop]
)

# 📊 Evaluate
loss, accuracy, auc = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Test Accuracy: {accuracy * 100:.2f}%")
print(f"✅ Test AUC Score: {auc:.4f}")

# 💾 Save Final Model
model.save("final_efficientnetb3_model.keras")
print("✅ Model training and saving completed successfully.")


Found 112 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 27s 1us/step


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4461 - auc: 0.4301 - loss: 0.7198 
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_efficientnetb3_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 46s 6s/step - accuracy: 0.4444 - auc: 0.4356 - loss: 0.7195 - val_accuracy: 0.5000 - val_auc: 0.2500 - val_loss: 0.7030
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4147 - auc: 0.3937 - loss: 0.7317
Epoch 2: val_accuracy did not improve from 0.50000
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.4211 - auc: 0.3974 - loss: 0.7290 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6960
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4875 - auc: 0.4487 - loss: 0.7128
Epoch 3: val_accuracy did not improve from 0.50000
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.4936 - auc: 0.4594 - loss: 0.7109 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4229 - auc: 0.3655 - loss: 0.7201
Epoch 4: val_accuracy did not improve from 0.50000
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.4222 - auc: 0.3698 - loss: 0.7197 - val_accuracy: 0.5000 - val_auc: 0.5417 -

4/4 ━━━━━━━━━━━━━━━━━━━━ 24s 6s/step - accuracy: 0.3928 - auc: 0.3932 - loss: 0.7332 - val_accuracy: 0.7500 - val_auc: 0.5799 - val_loss: 0.6916
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5890 - auc: 0.5223 - loss: 0.7050
Epoch 10: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.5890 - auc: 0.5267 - loss: 0.7033 - val_accuracy: 0.5000 - val_auc: 0.7083 - val_loss: 0.6964
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5708 - auc: 0.5203 - loss: 0.7060
Epoch 11: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.5692 - auc: 0.5217 - loss: 0.7062 - val_accuracy: 0.5000 - val_auc: 0.5799 - val_loss: 0.6957
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5155 - auc: 0.4520 - loss: 0.7179
Epoch 12: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step - accuracy: 0.5106 - auc: 0.4576 - loss: 0.7166 - val_accuracy: 0.5000 - val_auc: 0.